In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import math
import duckdb
import numpy as np
# Display all columns
pd.set_option('display.max_columns', None)

# Set max column width to a large value
pd.set_option('display.max_colwidth', None)

# Create a new DuckDB database file
con = duckdb.connect('my_database.duckdb')

In [2]:
import pandas as pd
import re

# Read the CSV file line by line
file_path = 'decoded_polygon.csv'
lines = []

with open(file_path, 'r') as file:
    lines = file.readlines()

# Initialize dictionaries to store lines for each transaction type
transaction_dict = {}

# Function to parse fields without combining text between single quotes
def parse_line(line):
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    return fields

# Iterate over each line to categorize it based on the 6th field
for line in lines:
    fields = parse_line(line.strip())
    if len(fields) > 5:  # Ensure there are at least 6 fields after cleaning
        transaction_type = fields[5]
        if transaction_type not in transaction_dict:
            transaction_dict[transaction_type] = []
        transaction_dict[transaction_type].append(fields)

# Create separate DataFrames for each transaction type
dataframes = {}
for transaction_type, data in transaction_dict.items():
    # Determine the number of columns for this transaction type
    num_columns = max(len(fields) for fields in data)
    # Pad the data with empty strings to ensure all rows have the same number of columns
    padded_data = [fields + [''] * (num_columns - len(fields)) for fields in data]
    # Create DataFrame
    df = pd.DataFrame(padded_data, columns=[f'Field_{i+1}' for i in range(num_columns)])
    dataframes[transaction_type] = df

# Print out the names of the created DataFrames
for transaction_type in dataframes.keys():
    print(f'Created DataFrame: {transaction_type}')


Created DataFrame: AuthorizerChanged
Created DataFrame: InternalBalanceChanged
Created DataFrame: PoolRegistered
Created DataFrame: TokensRegistered
Created DataFrame: PoolBalanceChanged
Created DataFrame: Swap
Created DataFrame: FlashLoan
Created DataFrame: RelayerApprovalChanged
Created DataFrame: PoolBalanceManaged
Created DataFrame: ExternalBalanceTransfer
Created DataFrame: TokensDeregistered


In [3]:
# Function to combine fields after Field_6 and before '0x'
def combine_fields(df):
    combined_data = []
    for index, row in df.iterrows():
        combined_row = row[:6].tolist()  # Get the first six fields
        remaining_fields = row[6:].tolist()  # Get the remaining fields
        combined_field = []
        for field in remaining_fields:
            if re.match(r'^0x', field):  # Check for '0x' pattern
                break
            combined_field.append(field)
        combined_row.append(' '.join(combined_field))
        combined_data.append(combined_row + remaining_fields[len(combined_field):])
    return pd.DataFrame(combined_data, columns=[f'Field_{i+1}' for i in range(len(combined_data[0]))])

dataframes['Swap'] = combine_fields(dataframes['Swap'])
dataframes['PoolRegistered'] = combine_fields(dataframes['PoolRegistered'])
dataframes['PoolBalanceChanged'] = combine_fields(dataframes['PoolBalanceChanged'])
dataframes['PoolBalanceManaged'] = combine_fields(dataframes['PoolBalanceManaged'])

In [4]:
Swap = dataframes['Swap']
PoolRegistered = dataframes['PoolRegistered']
PoolBalanceChanged = dataframes['PoolBalanceChanged']
PoolBalanceManaged = dataframes['PoolBalanceManaged']
ExternalBalanceTransfer = dataframes['ExternalBalanceTransfer']
FlashLoan = dataframes['FlashLoan']
PoolBalanceManaged

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14,Field_15,Field_16
0,24826461,57,296,0x03ce5a4fcadab65f6fbb9227f40e8844979baa0eb4d47fbffb9171ea4ea5bf89,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\x89\xfc\xb6\xf85\xa8\xa6\x93\xd6\xa7|\x0b[\xe8e\xf7\r\xc0FI\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x02\xcb',0xbEfCa036a376e260B4A16629E59e09344A516645,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,0,648000000000000000,None,None,None,None,None
1,24826461,57,297,0x03ce5a4fcadab65f6fbb9227f40e8844979baa0eb4d47fbffb9171ea4ea5bf89,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\x89\xfc\xb6\xf85\xa8\xa6\x93\xd6\xa7|\x0b[\xe8e\xf7\r\xc0FI\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x02\xcb',0xbEfCa036a376e260B4A16629E59e09344A516645,0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063,0,99000000000000000000,None,None,None,None,None
2,24826461,57,300,0x03ce5a4fcadab65f6fbb9227f40e8844979baa0eb4d47fbffb9171ea4ea5bf89,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\x89\xfc\xb6\xf85\xa8\xa6\x93\xd6\xa7|\x0b[\xe8e\xf7\r\xc0FI\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x02\xcb',0xbEfCa036a376e260B4A16629E59e09344A516645,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,648000000000000000,-648000000000000000,None,None,None,None,None
3,24826461,57,303,0x03ce5a4fcadab65f6fbb9227f40e8844979baa0eb4d47fbffb9171ea4ea5bf89,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\x89\xfc\xb6\xf85\xa8\xa6\x93\xd6\xa7|\x0b[\xe8e\xf7\r\xc0FI\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x02\xcb',0xbEfCa036a376e260B4A16629E59e09344A516645,0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063,99000000000000000000,-99000000000000000000,None,None,None,None,None
4,25294681,56,242,0x536f9623232b550b22075c7386c882e69e83fd7a25204e97a8ece750553ab586,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'\x89\xfc\xb6\xf85\xa8\xa6\x93\xd6\xa7|\x0b[\xe8e\xf7\r\xc0FI\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x02\xcb',0xbEfCa036a376e260B4A16629E59e09344A516645,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,0,2820575000000000000,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3853,54106389,94,436,0xc1d3fff402f26772573c73b630d202de60e0f536bdd92cf39735c54d5d6eb630,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'V\x9d\xc6\xa3@/\x97\x15%\x87R\xfe\xea\xa1\x0e\x07cx\xf1Z\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x0b\x83',0x3c14801DC6402E0560D69083F2b238b4C4b4DAfe,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,0,-69311684527391,None,None,None,None,None
3854,54107723,61,215,0xd416f3e633d1f22fe00a6cf85c5f70227fc6ea370b5c84e12d44ca14ba5c33fd,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'V\x9d\xc6\xa3@/\x97\x15%\x87R\xfe\xea\xa1\x0e\x07cx\xf1Z\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x0b\x83',0x3c14801DC6402E0560D69083F2b238b4C4b4DAfe,0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174,-213305761376,213305761376,None,None,None,None,None
3855,54107723,61,217,0xd416f3e633d1f22fe00a6cf85c5f70227fc6ea370b5c84e12d44ca14ba5c33fd,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'V\x9d\xc6\xa3@/\x97\x15%\x87R\xfe\xea\xa1\x0e\x07cx\xf1Z\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x0b\x83',0x3c14801DC6402E0560D69083F2b238b4C4b4DAfe,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,-28361807379466902545,28361807379466902545,None,None,None,None,None
3856,54107723,61,218,0xd416f3e633d1f22fe00a6cf85c5f70227fc6ea370b5c84e12d44ca14ba5c33fd,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,b'V\x9d\xc6\xa3@/\x97\x15%\x87R\xfe\xea\xa1\x0e\x07cx\xf1Z\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x0b\x83',0x3c14801DC6402E0560D69083F2b238b4C4b4DAfe,0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174,0,-213305761376,None,None,None,None,None


In [5]:
# Function to add double quotes around the value
def byte_to_hex(byte_string):
    # Convert the string representation of the byte string to an actual byte string
    byte_string = eval(byte_string)
    # Convert the byte string to a hexadecimal string
    return byte_string.hex()

# Apply the function to the Field_7 column
PoolBalanceChanged['Field_7'] = PoolBalanceChanged['Field_7'].apply(byte_to_hex)
PoolBalanceManaged['Field_7'] = PoolBalanceManaged['Field_7'].apply(byte_to_hex)
Swap['Field_7'] = Swap['Field_7'].apply(byte_to_hex)

In [6]:
PoolBalanceManaged

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14,Field_15,Field_16
0,24826461,57,296,0x03ce5a4fcadab65f6fbb9227f40e8844979baa0eb4d47fbffb9171ea4ea5bf89,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,89fcb6f835a8a693d6a77c0b5be865f70dc046490002000000000000000002cb,0xbEfCa036a376e260B4A16629E59e09344A516645,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,0,648000000000000000,None,None,None,None,None
1,24826461,57,297,0x03ce5a4fcadab65f6fbb9227f40e8844979baa0eb4d47fbffb9171ea4ea5bf89,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,89fcb6f835a8a693d6a77c0b5be865f70dc046490002000000000000000002cb,0xbEfCa036a376e260B4A16629E59e09344A516645,0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063,0,99000000000000000000,None,None,None,None,None
2,24826461,57,300,0x03ce5a4fcadab65f6fbb9227f40e8844979baa0eb4d47fbffb9171ea4ea5bf89,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,89fcb6f835a8a693d6a77c0b5be865f70dc046490002000000000000000002cb,0xbEfCa036a376e260B4A16629E59e09344A516645,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,648000000000000000,-648000000000000000,None,None,None,None,None
3,24826461,57,303,0x03ce5a4fcadab65f6fbb9227f40e8844979baa0eb4d47fbffb9171ea4ea5bf89,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,89fcb6f835a8a693d6a77c0b5be865f70dc046490002000000000000000002cb,0xbEfCa036a376e260B4A16629E59e09344A516645,0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063,99000000000000000000,-99000000000000000000,None,None,None,None,None
4,25294681,56,242,0x536f9623232b550b22075c7386c882e69e83fd7a25204e97a8ece750553ab586,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,89fcb6f835a8a693d6a77c0b5be865f70dc046490002000000000000000002cb,0xbEfCa036a376e260B4A16629E59e09344A516645,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,0,2820575000000000000,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3853,54106389,94,436,0xc1d3fff402f26772573c73b630d202de60e0f536bdd92cf39735c54d5d6eb630,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,569dc6a3402f9715258752feeaa10e076378f15a000200000000000000000b83,0x3c14801DC6402E0560D69083F2b238b4C4b4DAfe,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,0,-69311684527391,None,None,None,None,None
3854,54107723,61,215,0xd416f3e633d1f22fe00a6cf85c5f70227fc6ea370b5c84e12d44ca14ba5c33fd,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,569dc6a3402f9715258752feeaa10e076378f15a000200000000000000000b83,0x3c14801DC6402E0560D69083F2b238b4C4b4DAfe,0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174,-213305761376,213305761376,None,None,None,None,None
3855,54107723,61,217,0xd416f3e633d1f22fe00a6cf85c5f70227fc6ea370b5c84e12d44ca14ba5c33fd,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,569dc6a3402f9715258752feeaa10e076378f15a000200000000000000000b83,0x3c14801DC6402E0560D69083F2b238b4C4b4DAfe,0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619,-28361807379466902545,28361807379466902545,None,None,None,None,None
3856,54107723,61,218,0xd416f3e633d1f22fe00a6cf85c5f70227fc6ea370b5c84e12d44ca14ba5c33fd,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceManaged,569dc6a3402f9715258752feeaa10e076378f15a000200000000000000000b83,0x3c14801DC6402E0560D69083F2b238b4C4b4DAfe,0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174,0,-213305761376,None,None,None,None,None


In [7]:
PoolBalanceChanged[['Field_9', 'Field_10', 'Field_11', 'Field_12', 'Field_13', 'Field_14']].to_csv('PoolBalanceChanged.txt')

In [8]:
import pandas as pd
import re

# Read the text file
file_path = 'PoolBalanceChanged.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()

# Function to replace commas with dashes within square brackets
def replace_commas(text):
    return re.sub(r'\[([^\]]+)\]', lambda m: '[' + m.group(1).replace(',', '/') + ']', text)

# Apply the function to each line
modified_lines = [replace_commas(line) for line in lines]

# Save the modified lines back to a new text file (optional)
output_path = 'PoolBalanceChanged_modified.txt'
with open(output_path, 'w') as file:
    file.writelines(modified_lines)

# Create the pandas dataframe
PoolBalanceChanged_lists = pd.read_csv('PoolBalanceChanged_modified.txt')
PoolBalanceChanged_lists

,Unnamed: 0,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14
0,0,['0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6'/'0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619'/'0x85955046DF4668e1DD369D2DE9f3AEB98DD2A369'],[750277/129603959637633264/1030981029976757632],NaN,NaN,NaN,NaN
1,1,['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270'/'0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619'],[500000000000000000000/291484165989000000],[0/0],NaN,NaN,NaN
2,2,['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270'/'0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'/'0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619'/'0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3'],[265000000000000000000,297581484,NaN,NaN,NaN
3,3,['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'/'0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619'/'0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3'],[239398294/123414665257000000/44000000000000000000],NaN,NaN,NaN,NaN
4,4,['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'/'0x50B728D8D964fd00C2d0AAD81718b71311feF68a'/'0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39'/'0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619'/'0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3'/'0xD6DF932A45C0f255f85145f286eA0b292B21C90B'],NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
776132,776132,['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270'/'0xcd78A20c597E367A4e478a2411cEB790604D7c8F'/'0xfa68FB4628DFF1028CFEc22b4162FCcd0d45efb6'],[11388417787576058826/0/0],NaN,NaN,NaN,NaN
776133,776133,['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'/'0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063'],[-35159986/-37592936740532241313],[0/0],NaN,NaN,NaN
776134,776134,['0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39'/'0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619'/'0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3'/'0xD6DF932A45C0f255f85145f286eA0b292B21C90B'],[112448267758196050,0,NaN,NaN,NaN
776135,776135,['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'/'0x9C9e5fD8bbc25984B178FdCE6117Defa39d2db39'/'0xc2132D05D31c914a87C6611C10748AEb04B58e8F'],[-22251226/-24543624008800036/-126624608],NaN,NaN,NaN,NaN


In [9]:

# Function to convert the string to a list of strings
def convert_to_list(cell_value):
    if isinstance(cell_value, str):
        # Remove brackets
        cell_value = cell_value.strip('[]')
        # Split by '/'
        split_values = cell_value.split('/')
        # Convert to list of strings
        return [str(value) for value in split_values]
    else:
        return cell_value

# Apply the function to all columns
for column in ['Field_9', 'Field_10', 'Field_11']:
    PoolBalanceChanged_lists[column] = PoolBalanceChanged_lists[column].apply(convert_to_list)

PoolBalanceChanged_lists

,Unnamed: 0,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14
0,0,"['0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x85955046DF4668e1DD369D2DE9f3AEB98DD2A369']","[750277, 129603959637633264, 1030981029976757632]",NaN,NaN,NaN,NaN
1,1,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619']","[500000000000000000000, 291484165989000000]","[0, 0]",NaN,NaN,NaN
2,2,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3']",[265000000000000000000],[297581484],NaN,NaN,NaN
3,3,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3']","[239398294, 123414665257000000, 44000000000000000000]",NaN,NaN,NaN,NaN
4,4,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x50B728D8D964fd00C2d0AAD81718b71311feF68a', '0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3', '0xD6DF932A45C0f255f85145f286eA0b292B21C90B']",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
776132,776132,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0xcd78A20c597E367A4e478a2411cEB790604D7c8F', '0xfa68FB4628DFF1028CFEc22b4162FCcd0d45efb6']","[11388417787576058826, 0, 0]",NaN,NaN,NaN,NaN
776133,776133,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063']","[-35159986, -37592936740532241313]","[0, 0]",NaN,NaN,NaN
776134,776134,"['0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3', '0xD6DF932A45C0f255f85145f286eA0b292B21C90B']",[112448267758196050],[0],NaN,NaN,NaN
776135,776135,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x9C9e5fD8bbc25984B178FdCE6117Defa39d2db39', '0xc2132D05D31c914a87C6611C10748AEb04B58e8F']","[-22251226, -24543624008800036, -126624608]",NaN,NaN,NaN,NaN


In [10]:
PoolBalanceChanged['Field_9'] = PoolBalanceChanged_lists['Field_9']
PoolBalanceChanged['Field_10'] = PoolBalanceChanged_lists['Field_10']
PoolBalanceChanged['Field_11'] = PoolBalanceChanged_lists['Field_11']
PoolBalanceChanged = PoolBalanceChanged[[f'Field_{i}' for i in range(1, 12)]]
PoolBalanceChanged

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11
0,16032365,3,13,0x1620cb1bbd8eaa924a90d8bf74ae27db00528dda37a694567defb6d8dea718c4,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,647c1fd457b95b75d0972ff08fe01d7d7bda05df000100000000000000000001,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x85955046DF4668e1DD369D2DE9f3AEB98DD2A369']","[750277, 129603959637633264, 1030981029976757632]",NaN
1,16062824,181,217,0xaf9c4cebfa206a925f72c8fa3e79b283c30a96e374d04e6bfe4633dd4f833d20,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,61d5dc44849c9c87b0856a2a311536205c96c7fd000200000000000000000000,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619']","[500000000000000000000, 291484165989000000]","[0, 0]"
2,16068544,26,93,0x4f4e7c518d8e497b1a8dec197f3f52eb51b1a9ba598f6eb21fb2109eee580723,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,0297e37f1873d2dab4487aa67cd56b58e2f27875000100000000000000000002,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3']",[265000000000000000000],[297581484]
3,16068885,30,48,0x8440138cc452fad5828af10d2392c5cda5f469e4c77143bd96a1e0d18dd6149d,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,c6a5032dc4bf638e15b4a66bc718ba7ba474ff73000100000000000000000003,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3']","[239398294, 123414665257000000, 44000000000000000000]",NaN
4,16069221,3,32,0x697d57aa86e8ff7290c5d723889655a4f27389967bbf3dfce0b2f0956808db2f,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,3a19030ed746bd1c3f2b0f996ff9479af04c5f0a000100000000000000000004,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x50B728D8D964fd00C2d0AAD81718b71311feF68a', '0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3', '0xD6DF932A45C0f255f85145f286eA0b292B21C90B']",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
776132,57657147,75,313,0x0b70e1d86028e42213057d6709c581807db8c19a7de8f5796d685fbd922eef62,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,cd78a20c597e367a4e478a2411ceb790604d7c8f000000000000000000000c22,0xbAd3d774c81CF7C4D7d73400C815272cA4b47417,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0xcd78A20c597E367A4e478a2411cEB790604D7c8F', '0xfa68FB4628DFF1028CFEc22b4162FCcd0d45efb6']","[11388417787576058826, 0, 0]",NaN
776133,57657784,79,317,0x302f07e81af5534047512be7816221fb744503b5bb4ff347629693b38fa3884f,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,dac42eeb17758daa38caf9a3540c808247527ae3000200000000000000000a2b,0xD91613333480222Fd60bcDfC9A2EF9886d7E83A4,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063']","[-35159986, -37592936740532241313]","[0, 0]"
776134,57657842,31,106,0x5888a55181c9bb271791eafd4717bb4e96c11d622c90565cf39ed70d6579d216,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,ce66904b68f1f0703320bc631de7ee98b650b499000100000000000000000009,0x35b9eFedD2B80bD0E1988649AdF8AB463be3547E,"['0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3', '0xD6DF932A45C0f255f85145f286eA0b292B21C90B']",[112448267758196050],[0]
776135,57657863,42,555,0x154ed28460b21bb0f9150b62b63c6561c33e7603d9bb126f18ba2f85cdc9db8e,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,17f1ef81707811ea15d9ee7c741179bbe2a63887000100000000000000000799,0xD9161

In [11]:
PoolBalanceChanged

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11
0,16032365,3,13,0x1620cb1bbd8eaa924a90d8bf74ae27db00528dda37a694567defb6d8dea718c4,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,647c1fd457b95b75d0972ff08fe01d7d7bda05df000100000000000000000001,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x85955046DF4668e1DD369D2DE9f3AEB98DD2A369']","[750277, 129603959637633264, 1030981029976757632]",NaN
1,16062824,181,217,0xaf9c4cebfa206a925f72c8fa3e79b283c30a96e374d04e6bfe4633dd4f833d20,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,61d5dc44849c9c87b0856a2a311536205c96c7fd000200000000000000000000,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619']","[500000000000000000000, 291484165989000000]","[0, 0]"
2,16068544,26,93,0x4f4e7c518d8e497b1a8dec197f3f52eb51b1a9ba598f6eb21fb2109eee580723,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,0297e37f1873d2dab4487aa67cd56b58e2f27875000100000000000000000002,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3']",[265000000000000000000],[297581484]
3,16068885,30,48,0x8440138cc452fad5828af10d2392c5cda5f469e4c77143bd96a1e0d18dd6149d,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,c6a5032dc4bf638e15b4a66bc718ba7ba474ff73000100000000000000000003,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3']","[239398294, 123414665257000000, 44000000000000000000]",NaN
4,16069221,3,32,0x697d57aa86e8ff7290c5d723889655a4f27389967bbf3dfce0b2f0956808db2f,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,3a19030ed746bd1c3f2b0f996ff9479af04c5f0a000100000000000000000004,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x50B728D8D964fd00C2d0AAD81718b71311feF68a', '0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3', '0xD6DF932A45C0f255f85145f286eA0b292B21C90B']",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
776132,57657147,75,313,0x0b70e1d86028e42213057d6709c581807db8c19a7de8f5796d685fbd922eef62,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,cd78a20c597e367a4e478a2411ceb790604d7c8f000000000000000000000c22,0xbAd3d774c81CF7C4D7d73400C815272cA4b47417,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0xcd78A20c597E367A4e478a2411cEB790604D7c8F', '0xfa68FB4628DFF1028CFEc22b4162FCcd0d45efb6']","[11388417787576058826, 0, 0]",NaN
776133,57657784,79,317,0x302f07e81af5534047512be7816221fb744503b5bb4ff347629693b38fa3884f,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,dac42eeb17758daa38caf9a3540c808247527ae3000200000000000000000a2b,0xD91613333480222Fd60bcDfC9A2EF9886d7E83A4,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063']","[-35159986, -37592936740532241313]","[0, 0]"
776134,57657842,31,106,0x5888a55181c9bb271791eafd4717bb4e96c11d622c90565cf39ed70d6579d216,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,ce66904b68f1f0703320bc631de7ee98b650b499000100000000000000000009,0x35b9eFedD2B80bD0E1988649AdF8AB463be3547E,"['0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3', '0xD6DF932A45C0f255f85145f286eA0b292B21C90B']",[112448267758196050],[0]
776135,57657863,42,555,0x154ed28460b21bb0f9150b62b63c6561c33e7603d9bb126f18ba2f85cdc9db8e,0xBA12222222228d8Ba445958a75a0704d566BF2C8,PoolBalanceChanged,17f1ef81707811ea15d9ee7c741179bbe2a63887000100000000000000000799,0xD9161

In [12]:
# Select and rename specific columns
selected_columns = {'Field_1': 'block_number', 'Field_3': 'log_index', 'Field_4': 'transaction_hash', 'Field_6':'event', 'Field_7': 'pool_id', 'Field_8':'transaction_sender', 'Field_9': 'tokens', 'Field_10': 'delta', 'Field_11': 'protocol_fee'}
PoolBalanceChanged = PoolBalanceChanged[list(selected_columns.keys())].rename(columns=selected_columns)

selected_columns = {'Field_1': 'block_number', 'Field_3': 'log_index', 'Field_4': 'transaction_hash', 'Field_6':'event', 'Field_7': 'pool_id', 'Field_8':'transaction_sender', 'Field_9': 'token', 'Field_10': 'cash_delta'}
PoolBalanceManaged = PoolBalanceManaged[list(selected_columns.keys())].rename(columns=selected_columns)

selected_columns = {'Field_1': 'block_number', 'Field_3': 'log_index', 'Field_4': 'transaction_hash', 'Field_6':'event', 'Field_7': 'pool_id', 'Field_8':'token_in', 'Field_9': 'token_out', 'Field_10': 'amount_in', 'Field_11': 'amount_out'}
Swap = Swap[list(selected_columns.keys())].rename(columns=selected_columns)

PoolBalanceChanged

,block_number,log_index,transaction_hash,event,pool_id,transaction_sender,tokens,delta,protocol_fee
0,16032365,13,0x1620cb1bbd8eaa924a90d8bf74ae27db00528dda37a694567defb6d8dea718c4,PoolBalanceChanged,647c1fd457b95b75d0972ff08fe01d7d7bda05df000100000000000000000001,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x85955046DF4668e1DD369D2DE9f3AEB98DD2A369']","[750277, 129603959637633264, 1030981029976757632]",NaN
1,16062824,217,0xaf9c4cebfa206a925f72c8fa3e79b283c30a96e374d04e6bfe4633dd4f833d20,PoolBalanceChanged,61d5dc44849c9c87b0856a2a311536205c96c7fd000200000000000000000000,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619']","[500000000000000000000, 291484165989000000]","[0, 0]"
2,16068544,93,0x4f4e7c518d8e497b1a8dec197f3f52eb51b1a9ba598f6eb21fb2109eee580723,PoolBalanceChanged,0297e37f1873d2dab4487aa67cd56b58e2f27875000100000000000000000002,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3']",[265000000000000000000],[297581484]
3,16068885,48,0x8440138cc452fad5828af10d2392c5cda5f469e4c77143bd96a1e0d18dd6149d,PoolBalanceChanged,c6a5032dc4bf638e15b4a66bc718ba7ba474ff73000100000000000000000003,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3']","[239398294, 123414665257000000, 44000000000000000000]",NaN
4,16069221,32,0x697d57aa86e8ff7290c5d723889655a4f27389967bbf3dfce0b2f0956808db2f,PoolBalanceChanged,3a19030ed746bd1c3f2b0f996ff9479af04c5f0a000100000000000000000004,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x50B728D8D964fd00C2d0AAD81718b71311feF68a', '0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3', '0xD6DF932A45C0f255f85145f286eA0b292B21C90B']",NaN,NaN
...,...,...,...,...,...,...,...,...,...
776132,57657147,313,0x0b70e1d86028e42213057d6709c581807db8c19a7de8f5796d685fbd922eef62,PoolBalanceChanged,cd78a20c597e367a4e478a2411ceb790604d7c8f000000000000000000000c22,0xbAd3d774c81CF7C4D7d73400C815272cA4b47417,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0xcd78A20c597E367A4e478a2411cEB790604D7c8F', '0xfa68FB4628DFF1028CFEc22b4162FCcd0d45efb6']","[11388417787576058826, 0, 0]",NaN
776133,57657784,317,0x302f07e81af5534047512be7816221fb744503b5bb4ff347629693b38fa3884f,PoolBalanceChanged,dac42eeb17758daa38caf9a3540c808247527ae3000200000000000000000a2b,0xD91613333480222Fd60bcDfC9A2EF9886d7E83A4,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063']","[-35159986, -37592936740532241313]","[0, 0]"
776134,57657842,106,0x5888a55181c9bb271791eafd4717bb4e96c11d622c90565cf39ed70d6579d216,PoolBalanceChanged,ce66904b68f1f0703320bc631de7ee98b650b499000100000000000000000009,0x35b9eFedD2B80bD0E1988649AdF8AB463be3547E,"['0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3', '0xD6DF932A45C0f255f85145f286eA0b292B21C90B']",[112448267758196050],[0]
776135,57657863,555,0x154ed28460b21bb0f9150b62b63c6561c33e7603d9bb126f18ba2f85cdc9db8e,PoolBalanceChanged,17f1ef81707811ea15d9ee7c741179bbe2a63887000100000000000000000799,0xD91613333480222Fd60bcDfC9A2EF9886d7E83A4,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x9C9e5fD8bbc25984B178FdCE6117Defa39d2db39', '0xc2132D05D31c914a87C6611C10748AEb04B58e8F']","[-22251226, -24543624008800036, -126624608]",NaN


In [13]:
PoolBalanceChanged

,block_number,log_index,transaction_hash,event,pool_id,transaction_sender,tokens,delta,protocol_fee
0,16032365,13,0x1620cb1bbd8eaa924a90d8bf74ae27db00528dda37a694567defb6d8dea718c4,PoolBalanceChanged,647c1fd457b95b75d0972ff08fe01d7d7bda05df000100000000000000000001,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x85955046DF4668e1DD369D2DE9f3AEB98DD2A369']","[750277, 129603959637633264, 1030981029976757632]",NaN
1,16062824,217,0xaf9c4cebfa206a925f72c8fa3e79b283c30a96e374d04e6bfe4633dd4f833d20,PoolBalanceChanged,61d5dc44849c9c87b0856a2a311536205c96c7fd000200000000000000000000,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619']","[500000000000000000000, 291484165989000000]","[0, 0]"
2,16068544,93,0x4f4e7c518d8e497b1a8dec197f3f52eb51b1a9ba598f6eb21fb2109eee580723,PoolBalanceChanged,0297e37f1873d2dab4487aa67cd56b58e2f27875000100000000000000000002,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3']",[265000000000000000000],[297581484]
3,16068885,48,0x8440138cc452fad5828af10d2392c5cda5f469e4c77143bd96a1e0d18dd6149d,PoolBalanceChanged,c6a5032dc4bf638e15b4a66bc718ba7ba474ff73000100000000000000000003,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3']","[239398294, 123414665257000000, 44000000000000000000]",NaN
4,16069221,32,0x697d57aa86e8ff7290c5d723889655a4f27389967bbf3dfce0b2f0956808db2f,PoolBalanceChanged,3a19030ed746bd1c3f2b0f996ff9479af04c5f0a000100000000000000000004,0xd00ED528791F37892cdE39B7a7F0c3A3faf84C3f,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x50B728D8D964fd00C2d0AAD81718b71311feF68a', '0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3', '0xD6DF932A45C0f255f85145f286eA0b292B21C90B']",NaN,NaN
...,...,...,...,...,...,...,...,...,...
776132,57657147,313,0x0b70e1d86028e42213057d6709c581807db8c19a7de8f5796d685fbd922eef62,PoolBalanceChanged,cd78a20c597e367a4e478a2411ceb790604d7c8f000000000000000000000c22,0xbAd3d774c81CF7C4D7d73400C815272cA4b47417,"['0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', '0xcd78A20c597E367A4e478a2411cEB790604D7c8F', '0xfa68FB4628DFF1028CFEc22b4162FCcd0d45efb6']","[11388417787576058826, 0, 0]",NaN
776133,57657784,317,0x302f07e81af5534047512be7816221fb744503b5bb4ff347629693b38fa3884f,PoolBalanceChanged,dac42eeb17758daa38caf9a3540c808247527ae3000200000000000000000a2b,0xD91613333480222Fd60bcDfC9A2EF9886d7E83A4,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063']","[-35159986, -37592936740532241313]","[0, 0]"
776134,57657842,106,0x5888a55181c9bb271791eafd4717bb4e96c11d622c90565cf39ed70d6579d216,PoolBalanceChanged,ce66904b68f1f0703320bc631de7ee98b650b499000100000000000000000009,0x35b9eFedD2B80bD0E1988649AdF8AB463be3547E,"['0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39', '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', '0x9a71012B13CA4d3D0Cdc72A177DF3ef03b0E76A3', '0xD6DF932A45C0f255f85145f286eA0b292B21C90B']",[112448267758196050],[0]
776135,57657863,555,0x154ed28460b21bb0f9150b62b63c6561c33e7603d9bb126f18ba2f85cdc9db8e,PoolBalanceChanged,17f1ef81707811ea15d9ee7c741179bbe2a63887000100000000000000000799,0xD91613333480222Fd60bcDfC9A2EF9886d7E83A4,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x9C9e5fD8bbc25984B178FdCE6117Defa39d2db39', '0xc2132D05D31c914a87C6611C10748AEb04B58e8F']","[-22251226, -24543624008800036, -126624608]",NaN


In [15]:
con.execute('CREATE TABLE polygon_swap AS SELECT * FROM Swap').fetchdf()
con.execute('CREATE TABLE polygon_poolregistered AS SELECT * FROM PoolRegistered').fetchdf()
con.execute('CREATE TABLE polygon_poolbalancechanged AS SELECT * FROM PoolBalanceChanged').fetchdf()
con.execute('CREATE TABLE polygon_externalbalancetransfer AS SELECT * FROM ExternalBalanceTransfer').fetchdf()
con.execute('CREATE TABLE polygon_flashloan AS SELECT * FROM FlashLoan').fetchdf()
con.execute('CREATE TABLE polygon_poolbalancemanaged AS SELECT * FROM PoolBalanceManaged').fetchdf()

# Commit the changes
con.commit()

# Close the connection
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [17]:
PoolBalanceChanged[PoolBalanceChanged['pool_id']== 'ae8f935830f6b418804836eacb0243447b6d977c000200000000000000000ad1']  

,block_number,log_index,transaction_hash,event,pool_id,transaction_sender,tokens,delta,protocol_fee
688830,40156217,136,0xae27fed433feac9a4eb8a73871fbf307de107255fdb0d0fa289c81d065adb8a6,PoolBalanceChanged,ae8f935830f6b418804836eacb0243447b6d977c000200000000000000000ad1,0x62DE034b1A69eF853c9d0D8a33D26DF5cF26682E,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x385Eeac5cB85A38A9a07A70c73e0a3271CfB54A7']","[926021934197, 2580000000000000000000000]","[0, 0]"
689387,40192215,112,0xe73f87cd2d0ada69aca03439df8c5ca510f613297519c325526b05e4b5f62bdc,PoolBalanceChanged,ae8f935830f6b418804836eacb0243447b6d977c000200000000000000000ad1,0x512e06268d792A9eF842df83cAFf9FeBC775C427,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x385Eeac5cB85A38A9a07A70c73e0a3271CfB54A7']","[7530228, 22253593373290135604]","[0, 0]"
689388,40192277,5,0x3d18c85c7c9332dc53c54be8b85c84697c686b096dfbc923c96e63b6eb0bb85b,PoolBalanceChanged,ae8f935830f6b418804836eacb0243447b6d977c000200000000000000000ad1,0x512e06268d792A9eF842df83cAFf9FeBC775C427,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x385Eeac5cB85A38A9a07A70c73e0a3271CfB54A7']","[-7530228, -22253593282213992444]","[0, 0]"
689989,40228128,788,0x4755b6b0eaf0b5987d9e16033098cf692beb811f5e695e9eec4e9e9f8d7a99c9,PoolBalanceChanged,ae8f935830f6b418804836eacb0243447b6d977c000200000000000000000ad1,0xd5dF46dC1fCdF13f9067D6668ed411d5342d9DDC,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x385Eeac5cB85A38A9a07A70c73e0a3271CfB54A7']","[20000000, 0]","[0, 0]"
691370,40339116,451,0xdc219f35b25e4d3ba51afb0e1e308f08859a9de2b11255817b2eee48e2840984,PoolBalanceChanged,ae8f935830f6b418804836eacb0243447b6d977c000200000000000000000ad1,0xACCF791DeD07AB38b66b6e8f241d0625F3F3f0CB,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x385Eeac5cB85A38A9a07A70c73e0a3271CfB54A7']","[128149, 400000000000000000]","[0, 0]"
...,...,...,...,...,...,...,...,...,...
775722,57452809,204,0x23363f08ec8d720857741e1c3eb349fc59ebc4e4311310421a7779d3e2bdb18c,PoolBalanceChanged,ae8f935830f6b418804836eacb0243447b6d977c000200000000000000000ad1,0xe9Ea9cAB6426e7F001C9669B94176C94EeB901E5,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x385Eeac5cB85A38A9a07A70c73e0a3271CfB54A7']","[146048, 0]","[0, 0]"
775726,57453244,179,0xd07107785c8710cc9dcfbc66553ef06ac2b13e08c4d63a03adaa79c68e7e0f00,PoolBalanceChanged,ae8f935830f6b418804836eacb0243447b6d977c000200000000000000000ad1,0x9e5D6427D2cdaDC68870197b099C2Df535Ec3c97,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x385Eeac5cB85A38A9a07A70c73e0a3271CfB54A7']","[-87868632, -224221307445642693714]","[0, 0]"
775787,57483953,204,0x1d84bbb45c33c4f3cb7a081c06ddac3e65812e16e5cba65383fbf3329a13f3dd,PoolBalanceChanged,ae8f935830f6b418804836eacb0243447b6d977c000200000000000000000ad1,0xE04cE52a9055212201e8f044870A201aee1281C2,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x385Eeac5cB85A38A9a07A70c73e0a3271CfB54A7']","[48820251, 0]","[0, 0]"
775788,57484123,83,0x11e6284b657570c9e5c0df70fa3837554d612d56b07164030471dce9c0ef2ef0,PoolBalanceChanged,ae8f935830f6b418804836eacb0243447b6d977c000200000000000000000ad1,0xE04cE52a9055212201e8f044870A201aee1281C2,"['0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', '0x385Eeac5cB85A38A9a07A70c73e0a3271CfB54A7']","[-48398724, 0]","[0, 0]"
